# Метод аддитивной оптимизации

In [1]:
import pandas as pd
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
import numpy as np

## Заведение исходных данных

### Получение весов из работы 1,11 (экспертные методы)  
  
тут обязательно переименовываем столбцы так, чтобы названия соответствовали в этой таблице и в таблице с параметрами

In [2]:
weights = pd.read_csv('Искомый коэффициент значимости.csv',sep=';').set_index('Unnamed: 0').T
weights.rename(columns = {'стоимость (тыс. руб.)' : 'Стоимость, тыс.руб.',
                'площадь (м2)' : 'Площадь, м2',
                'количество комнат' : 'Количество комнат',
                'этаж' : 'Этаж',
                'материал стен': 'Материал'},
              inplace = True)
weights

Unnamed: 0,Район,"Стоимость, тыс.руб.","Площадь, м2",Количество комнат,Этаж,Материал,Сумма
Искомый коэффициент значимости,0.233333,0.22381,0.190476,0.07619,0.085714,0.190476,1.0


### Получение текущих данных

сразу добавляем строку с весами критериев, полученную пунктом выше

In [3]:
data = StringIO('''Адрес;Район;Количество комнат;Площадь, м2;Этаж;Материал;Стоимость, тыс.руб.
Каслинская ул, д. 99в;Калининский;3;100;3/11;Кирпич;8600
Шагольская ул, д. 34;Курчатовский;4;89;4/5;Кирпич;2499
Шагольская ул, д. 41;Курчатовский;3;65;2/5;Панельный;2150
Прокатная ул, д. 26;Металлургический;3;91;1/16;Панельный;2250
Свободы ул, д. 88б;Советский;3;50;6/6;Кирпич;4550
Лобинский 10-й пер, д. 3;Тракторозаводский;3;63;4/5;Панельный;2180
МОПРа площадь ул, д. 9;Центральный;4;198.1;9/16;Кирпич;8000''')
df = pd.read_csv(data, sep=";").set_index('Адрес')
originalDataLen = len(df)
df.loc['Веса критериев'] = weights.iloc[0]
df.head(originalDataLen+1)

,Район,Количество комнат,"Площадь, м2",Этаж,Материал,"Стоимость, тыс.руб."
Адрес,,,,,,
"Каслинская ул, д. 99в",Калининский,3.00000,100.000000,3/11,Кирпич,8600.00000
"Шагольская ул, д. 34",Курчатовский,4.00000,89.000000,4/5,Кирпич,2499.00000
"Шагольская ул, д. 41",Курчатовский,3.00000,65.000000,2/5,Панельный,2150.00000
"Прокатная ул, д. 26",Металлургический,3.00000,91.000000,1/16,Панельный,2250.00000
"Свободы ул, д. 88б",Советский,3.00000,50.000000,6/6,Кирпич,4550.00000
"Лобинский 10-й пер, д. 3",Тракторозаводский,3.00000,63.000000,4/5,Панельный,2180.00000
"МОПРа площадь ул, д. 9",Центральный,4.00000,198.100000,9/16,Кирпич,8000.00000
Веса критериев,0.233333,0.07619,0.190476,0.085714,0.190476,0.22381


### Замена буквенных столбцов на числовые эквиваленты

Тут необходимо определиться насколько важен, например, материал дома. Если кирпичный лучше панельного, то вместо "кирпичный" ставим 1, вместо "панельный" - 2. <b>И не забываем, что будем минимизировать данный столбец в последующем.
    
#### Интересный вопрос возник при оценке этажности. Если параметров много, то какие-то два параметра могут быть ближе друг к другу, чем другие. Например 3/11 и 9/16 не одно и то же, но ближе, чем 9/16 и 6/6. Как соблюсти пропорции при расставлении рангов в таком случае?

In [4]:
df.dtypes

Район                   object
Количество комнат      float64
Площадь, м2            float64
Этаж                    object
Материал                object
Стоимость, тыс.руб.    float64
dtype: object

In [5]:
df.replace(inplace = True,
          to_replace = {
              'Центральный' : 1,
              'Калининский' : 2,
              'Курчатовский' : 3,
              'Металлургический' : 4,
              'Тракторозаводский' : 5,
              'Советский' : 6
          })
df.replace(inplace = True,
          to_replace = {
              'Кирпич' : 1,
              'Панельный' : 2
          })
df.replace(inplace = True,
          to_replace = {
              '3/11' : 1,
              '9/16' : 1,
              '4/5' : 2,
              '4/5' : 2,
              '2/5' : 3,
              '1/16' : 3,
              '6/6' : 4,
          })
# 1-2 этаж я не хочу, будет шум
# высокий этаж без лифта (в 5-6 этажках) я тоже не хочу
# последний этаж тоже не хочу, будет крыша протекать
df

,Район,Количество комнат,"Площадь, м2",Этаж,Материал,"Стоимость, тыс.руб."
Адрес,,,,,,
"Каслинская ул, д. 99в",2.000000,3.00000,100.000000,1.000000,1.000000,8600.00000
"Шагольская ул, д. 34",3.000000,4.00000,89.000000,2.000000,1.000000,2499.00000
"Шагольская ул, д. 41",3.000000,3.00000,65.000000,3.000000,2.000000,2150.00000
"Прокатная ул, д. 26",4.000000,3.00000,91.000000,3.000000,2.000000,2250.00000
"Свободы ул, д. 88б",6.000000,3.00000,50.000000,4.000000,1.000000,4550.00000
"Лобинский 10-й пер, д. 3",5.000000,3.00000,63.000000,2.000000,2.000000,2180.00000
"МОПРа площадь ул, д. 9",1.000000,4.00000,198.100000,1.000000,1.000000,8000.00000
Веса критериев,0.233333,0.07619,0.190476,0.085714,0.190476,0.22381


## Определим максимум и минимум каждого локального критерия

In [6]:
df.loc['max'] = (df.head(5).max(axis=0))
df.loc['min'] = (df.head(5).min(axis=0))

In [7]:
df.head(3+originalDataLen)

,Район,Количество комнат,"Площадь, м2",Этаж,Материал,"Стоимость, тыс.руб."
Адрес,,,,,,
"Каслинская ул, д. 99в",2.000000,3.00000,100.000000,1.000000,1.000000,8600.00000
"Шагольская ул, д. 34",3.000000,4.00000,89.000000,2.000000,1.000000,2499.00000
"Шагольская ул, д. 41",3.000000,3.00000,65.000000,3.000000,2.000000,2150.00000
"Прокатная ул, д. 26",4.000000,3.00000,91.000000,3.000000,2.000000,2250.00000
"Свободы ул, д. 88б",6.000000,3.00000,50.000000,4.000000,1.000000,4550.00000
"Лобинский 10-й пер, д. 3",5.000000,3.00000,63.000000,2.000000,2.000000,2180.00000
"МОПРа площадь ул, д. 9",1.000000,4.00000,198.100000,1.000000,1.000000,8000.00000
Веса критериев,0.233333,0.07619,0.190476,0.085714,0.190476,0.22381
max,6.000000,4.00000,100.000000,4.000000,2.000000,8600.00000


### Выбор критериев которые максимизируем и минимизируем  
  
Правило выбора таково:  
     1. если критерий имеет прямую зависимость с качеством товара <b>(критерий растет - качество растет), то МАКСИМИЗИРУЕМ</b>  
     2. если критерий имеет обратную зависимость с качеством товара <b>(критерий падает - качество растёт), то МИНИМИЗИРУЕМ</b>  

In [ ]:
maximizationMinimizationParametrDict = {}
print('"+" - максимизируем; "-" - минимизируем.')
for columnName in df.columns:
    maximizationMinimizationParametrDict[columnName] = \
        input(f'Критерий: "{columnName}", введите +/-: ')

"+" - максимизируем; "-" - минимизируем.
Критерий: "Район", введите +/-: -
Критерий: "Количество комнат", введите +/-: +
Критерий: "Площадь, м2", введите +/-: +
Критерий: "Этаж", введите +/-: -


In [ ]:
maximizationMinimizationParametrDict

## Максимизируем/минимизируем столбцы двумя типами

In [ ]:
normalizedType1DF = df.copy()
normalizedType2DF = df.copy()
for columnName in df.columns:
    
    denominator = df[columnName]['max'] - df[columnName]['min']
    
    if maximizationMinimizationParametrDict[columnName] == '+':
        normalizedType1DF[columnName].iloc[0:originalDataLen] = \
            df[columnName].iloc[0:originalDataLen] / df[columnName]['max']
        
        normalizedType2DF[columnName].iloc[0:originalDataLen] = \
            (df[columnName].iloc[0:originalDataLen] - df[columnName]['min']) / denominator
    else:
        normalizedType1DF[columnName].iloc[0:originalDataLen] = \
            1 - df[columnName].iloc[0:originalDataLen] / df[columnName]['max']
        
        normalizedType2DF[columnName].iloc[0:originalDataLen] = \
            (df[columnName]['max'] - df[columnName].iloc[0:originalDataLen]) / denominator


### Типом 1: текущее / максимальное; 1 - текущее / максимальное

In [ ]:
normalizedType1DF.head(originalDataLen+1)

### Типом 2: ( текущее - мин ) / ( макс - мин ); ( макс - текущее  ) / ( макс - мин ) 

In [ ]:
normalizedType2DF.head(originalDataLen)

In [ ]:
normalizedType1DF['Обобщенная функ. цели Fi'] = \
    (normalizedType1DF[:][0:originalDataLen]*normalizedType1DF[:].loc['Веса критериев']).sum(axis=1)
print('Fi max =',normalizedType1DF['Обобщенная функ. цели Fi'].max())

In [ ]:
normalizedType1DF.head(originalDataLen+1)

In [ ]:
normalizedType2DF['Обобщенная функ. цели Fi'] = \
    (normalizedType2DF[:][0:originalDataLen]*normalizedType2DF[:].loc['Веса критериев']).sum(axis=1)
print('Fi max =',normalizedType2DF['Обобщенная функ. цели Fi'].max())

In [ ]:
normalizedType2DF.head(originalDataLen+1)